In [51]:
# src/data_processing.py

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer

In [52]:
def load_data(file_path):
    return pd.read_csv(file_path)

In [53]:
def create_aggregate_features(df):
    grouped = df.groupby("CustomerId")["Amount"]
    df_agg = pd.DataFrame({'CustomerId': grouped.groups.keys()})
    df_agg.set_index('CustomerId', inplace=True)
    df_agg["Total_Transaction_Amount"] = grouped.sum()
    df_agg["avg_transaction_amount"] = grouped.mean()
    df_agg["transaction_count"] = grouped.count()
    df_agg["std_transaction_amount"] = grouped.std()
    return df_agg.reset_index()

In [54]:
def extract_datetime_features(df):
    df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])
    df['transaction_hour'] = df['TransactionStartTime'].dt.hour
    df['transaction_day'] = df['TransactionStartTime'].dt.day
    df['transaction_month'] = df['TransactionStartTime'].dt.month
    df['transaction_year'] = df['TransactionStartTime'].dt.year
    return df

In [55]:
def one_hot_encode(df, columns):
    encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    encoded_array = encoder.fit_transform(df[columns])
    encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(columns))
    df = pd.concat([df.drop(columns=columns).reset_index(drop=True), encoded_df], axis=1)
    return df

In [56]:
def impute_missing(df):
    numerical_columns = df.select_dtypes(include=['int64', 'float64'])
    num_imputer = SimpleImputer(strategy="median")
    for column in numerical_columns.columns:
        df[column] = num_imputer.fit_transform(df[[column]]).ravel()
    return df

In [57]:
def normalize(df):
    numerical_columns = df.select_dtypes(include=['int64', 'float64'])
    normalizer = MinMaxScaler()
    for column in numerical_columns.columns:
        df[column] = normalizer.fit_transform(df[[column]])
    return df

In [58]:
def standardize(df):
    numerical_columns = df.select_dtypes(include=['int64', 'float64'])
    scaler = StandardScaler()
    for column in numerical_columns.columns:
        df[column] = scaler.fit_transform(df[[column]])
    return df

In [59]:
address = "C:/Users/hp/Desktop/Kifya/Week_5/credit-risk-model/data/"
df = load_data(address + "processed/eda_processed.csv")
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Recency,Frequency,Monetary
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,83.0,1.0,10000.0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0,83.0,1.0,10000.0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0,89.0,5.0,30400.0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,25.0,11.0,4775.0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0,11.0,6.0,32000.0


In [60]:
df_agg = create_aggregate_features(df)
df_agg.head()

,CustomerId,Total_Transaction_Amount,avg_transaction_amount,transaction_count,std_transaction_amount
0,CustomerId_1,-10000.0,-10000.000000,1,NaN
1,CustomerId_10,-10000.0,-10000.000000,1,NaN
2,CustomerId_1001,20000.0,4000.000000,5,6558.963333
3,CustomerId_1002,4225.0,384.090909,11,560.498966
4,CustomerId_1003,20000.0,3333.333333,6,6030.478146


In [61]:
df = extract_datetime_features(df)
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Recency,Frequency,Monetary,transaction_hour,transaction_day,transaction_month,transaction_year
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,83.0,1.0,10000.0,2,15,11,2018
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0,83.0,1.0,10000.0,2,15,11,2018
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0,89.0,5.0,30400.0,2,15,11,2018
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,25.0,11.0,4775.0,3,15,11,2018
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0,11.0,6.0,32000.0,3,15,11,2018


In [62]:
df = impute_missing(df)
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Recency,Frequency,Monetary,transaction_hour,transaction_day,transaction_month,transaction_year
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256.0,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000.0,2018-11-15 02:18:49+00:00,2.0,0.0,83.0,1.0,10000.0,2,15,11,2018
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256.0,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20.0,2018-11-15 02:19:08+00:00,2.0,0.0,83.0,1.0,10000.0,2,15,11,2018
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256.0,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500.0,2018-11-15 02:44:21+00:00,2.0,0.0,89.0,5.0,30400.0,2,15,11,2018
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256.0,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800.0,2018-11-15 03:32:55+00:00,2.0,0.0,25.0,11.0,4775.0,3,15,11,2018
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256.0,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644.0,2018-11-15 03:34:21+00:00,2.0,0.0,11.0,6.0,32000.0,3,15,11,2018


In [63]:
df = one_hot_encode(df, ['ProductCategory', 'ChannelId'])
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Recency,Frequency,Monetary,transaction_hour,transaction_day,transaction_month,transaction_year,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_1,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256.0,ProviderId_6,ProductId_10,1000.0,1000.0,2018-11-15 02:18:49+00:00,2.0,0.0,83.0,1.0,10000.0,2,15,11,2018,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256.0,ProviderId_4,ProductId_6,-20.0,20.0,2018-11-15 02:19:08+00:00,2.0,0.0,83.0,1.0,10000.0,2,15,11,2018,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256.0,ProviderId_6,ProductId_1,500.0,500.0,2018-11-15 02:44:21+00:00,2.0,0.0,89.0,5.0,30400.0,2,15,11,2018,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256.0,ProviderId_1,ProductId_21,20000.0,21800.0,2018-11-15 03:32:55+00:00,2.0,0.0,25.0,11.0,4775.0,3,15,11,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256.0,ProviderId_4,ProductId_6,-644.0,644.0,2018-11-15 03:34:21+00:00,2.0,0.0,11.0,6.0,32000.0,3,15,11,2018,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [64]:
df = normalize(df)
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Recency,Frequency,Monetary,transaction_hour,transaction_day,transaction_month,transaction_year,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_1,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,0.0,ProviderId_6,ProductId_10,0.092004,0.000101,2018-11-15 02:18:49+00:00,0.5,0.0,0.922222,0.000000,0.000095,2,15,11,2018,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,0.0,ProviderId_4,ProductId_6,0.091910,0.000002,2018-11-15 02:19:08+00:00,0.5,0.0,0.922222,0.000000,0.000095,2,15,11,2018,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,0.0,ProviderId_6,ProductId_1,0.091958,0.000050,2018-11-15 02:44:21+00:00,0.5,0.0,0.988889,0.000978,0.000289,2,15,11,2018,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,0.0,ProviderId_1,ProductId_21,0.093750,0.002206,2018-11-15 03:32:55+00:00,0.5,0.0,0.277778,0.002445,0.000045,3,15,11,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,0.0,ProviderId_4,ProductId_6,0.091853,0.000065,2018-11-15 03:34:21+00:00,0.5,0.0,0.122222,0.001222,0.000305,3,15,11,2018,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [65]:
df = standardize(df)
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Recency,Frequency,Monetary,transaction_hour,transaction_day,transaction_month,transaction_year,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_1,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,0.0,ProviderId_6,ProductId_10,-0.046371,-0.072291,2018-11-15 02:18:49+00:00,-0.349252,-0.044962,10.667317,-0.344877,-0.056885,2,15,11,2018,1.060447,-0.13096,-0.950503,-0.04281,-0.004572,-0.047572,-0.016168,-0.116409,-0.143114,-0.075205,-0.796656,0.824740,-0.105245
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,0.0,ProviderId_4,ProductId_6,-0.054643,-0.080251,2018-11-15 02:19:08+00:00,-0.349252,-0.044962,10.667317,-0.344877,-0.056885,2,15,11,2018,-0.942999,-0.13096,1.052074,-0.04281,-0.004572,-0.047572,-0.016168,-0.116409,-0.143114,-0.075205,1.255247,-1.212503,-0.105245
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,0.0,ProviderId_6,ProductId_1,-0.050426,-0.076352,2018-11-15 02:44:21+00:00,-0.349252,-0.044962,11.756797,-0.139782,-0.019022,2,15,11,2018,1.060447,-0.13096,-0.950503,-0.04281,-0.004572,-0.047572,-0.016168,-0.116409,-0.143114,-0.075205,-0.796656,0.824740,-0.105245
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,0.0,ProviderId_1,ProductId_21,0.107717,0.096648,2018-11-15 03:32:55+00:00,-0.349252,-0.044962,0.135687,0.167861,-0.066583,3,15,11,2018,-0.942999,-0.13096,-0.950503,-0.04281,-0.004572,-0.047572,-0.016168,-0.116409,6.987414,-0.075205,-0.796656,0.824740,-0.105245
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,0.0,ProviderId_4,ProductId_6,-0.059704,-0.075183,2018-11-15 03:34:21+00:00,-0.349252,-0.044962,-2.406431,-0.088508,-0.016053,3,15,11,2018,-0.942999,-0.13096,1.052074,-0.04281,-0.004572,-0.047572,-0.016168,-0.116409,-0.143114,-0.075205,1.255247,-1.212503,-0.105245


In [66]:
df.to_csv(address + "processed/processed_data_feature_eng.csv", index=False)